In [1]:
from grandma_stock_valuation import FileLogger, YahooDataLoader, batchValuation, addCashPortfolio, allocatePortfolio
import time

logger = FileLogger()
logPrint = logger.log_pandas


### Refresh data of each instrument

Query data of the selected instruments from Yahoo Finance.

With `save=True`, a `__data__` folder will be created to store the queried data.

In [2]:
d_instrument = {
    'IVV':'SP500',
    '3073.HK':'Greater China',
    'VPL':'Developed Asia-Pacific',
    'IEV':'Europe',
    'ASEA':'SE Asia',

    'AAXJ':'Asia ex Japan',
    'EZU':'Eurozone'
}

d_instrument_data = {}
for ticker, name in d_instrument.items():
    logPrint(f"{ticker}: {name}")

    # Refresh data
    yahoo = YahooDataLoader(ticker, date_start='2012-01-01', verbose=0, printfunc=logPrint) # data_start can be removed, if this instrument already has historical data under __data__ folder.
    df = yahoo.queryEOD(save=True)

    d_instrument_data[ticker] = df
    time.sleep(2)



2022-02-18 18:19:58,124 INFO IVV: SP500
2022-02-18 18:20:00,436 INFO 3073.HK: Greater China
2022-02-18 18:20:02,681 INFO VPL: Developed Asia-Pacific
2022-02-18 18:20:04,951 INFO IEV: Europe
2022-02-18 18:20:07,252 INFO ASEA: SE Asia
2022-02-18 18:20:09,474 INFO AAXJ: Asia ex Japan
2022-02-18 18:20:11,700 INFO EZU: Eurozone


### Valuate the instruments

With `save_result=True`, a `__output__` folder will be created to store valuation metrics in a .csv file.

A figure of each instrument, which is the price chart with fitted trend line, will be stored under `__output__/images`.

In [3]:
df_valuation_metrics, d_fig = batchValuation(d_instrument_data, verbose=0, save_result=True, printfunc=logPrint)

df_valuation_metrics

,ticker,r2_train,train_years,annualized_return,currenct_price,fair_price,over_value_range,over_value_years
0,IVV,0.970875,9.99726,0.134298,439.100006,410.22553,0.070387,0.524109
1,3073.HK,0.916884,10.00274,0.082724,51.220001,52.423557,-0.022958,-0.18992
2,VPL,0.890417,9.99726,0.067471,75.190002,77.105931,-0.024848,-0.167653
3,IEV,0.780835,9.99726,0.052748,52.220001,49.172989,0.061965,1.174748
4,ASEA,0.345249,9.99726,0.016434,16.01,14.345752,0.11601,7.059128
5,AAXJ,0.850795,9.99726,0.061776,82.010002,81.353659,0.008068,0.130597
6,EZU,0.796432,9.99726,0.061354,46.779999,45.647313,0.024814,0.404439


### Construct a portfolio

Based on the valuation, the suggested allocation of the insturments in your portfolio.

In [12]:
instruments_select = ['IVV', '3073.HK', 'VPL', 'IEV', 'ASEA']

df_portfolio = df_valuation_metrics[df_valuation_metrics['ticker'].isin(instruments_select)].copy()

df_portfolio['portfolio_allocation'] = allocatePortfolio(df_portfolio['over_value_years'], with_cash=False)

df_portfolio

,ticker,r2_train,train_years,annualized_return,currenct_price,fair_price,over_value_range,over_value_years,portfolio_allocation
0,IVV,0.970875,9.99726,0.134298,439.100006,410.22553,0.070387,0.524109,0.133116
1,3073.HK,0.916884,10.00274,0.082724,51.220001,52.423557,-0.022958,-0.18992,0.417241
2,VPL,0.890417,9.99726,0.067471,75.190002,77.105931,-0.024848,-0.167653,0.402637
3,IEV,0.780835,9.99726,0.052748,52.220001,49.172989,0.061965,1.174748,0.047002
4,ASEA,0.345249,9.99726,0.016434,16.01,14.345752,0.11601,7.059128,0.000004


Optionally, you can add cash as part of your portfolio. This allows the model to realize profit when most instruments are over-valued.

In [13]:
instruments_select = ['IVV', '3073.HK', 'VPL', 'IEV', 'ASEA']

df_portfolio = df_valuation_metrics[df_valuation_metrics['ticker'].isin(instruments_select)].copy()

df_portfolio_with_cash = addCashPortfolio(df_portfolio)

df_portfolio_with_cash['portfolio_allocation'] = allocatePortfolio(df_portfolio_with_cash['over_value_years'], with_cash=True)

df_portfolio_with_cash

,ticker,r2_train,train_years,annualized_return,currenct_price,fair_price,over_value_range,over_value_years,portfolio_allocation
0,IVV,0.970875,9.99726,0.134298,439.100006,410.22553,0.070387,0.524109,0.101778
1,3073.HK,0.916884,10.00274,0.082724,51.220001,52.423557,-0.022958,-0.189920,0.319015
2,VPL,0.890417,9.99726,0.067471,75.190002,77.105931,-0.024848,-0.167653,0.307849
3,IEV,0.780835,9.99726,0.052748,52.220001,49.172989,0.061965,1.174748,0.035937
4,ASEA,0.345249,9.99726,0.016434,16.01,14.345752,0.11601,7.059128,0.000003
5,cash,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.235419
